<a href="https://colab.research.google.com/github/ozdemirfadime/python/blob/master/BusinessCase_Telecom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Telecom Churn Analysis

## Custumer Churn 
It is when an existing customer, user, subscriber, or any kind of return client stops doing business or ends the relationship with a company.
Every business depends on customer's loyalty. The repeat business from customer is one of the cornerstone for business profitability. So it is important to know the reason of customers leaving a business. Customers going away is known as customer churn.

### Types of Customer Churn  

- Contractual Churn : When a customer is under a contract for a service and decides to cancel the service e.g. Cable TV, SaaS.
Voluntary Churn : When a user voluntarily cancels a service e.g. Cellular connection.
- Non-Contractual Churn : When a customer is not under a contract for a service and decides to cancel the service e.g. Consumer Loyalty in retail stores.
- Involuntary Churn : When a churn occurs without any request of the customer e.g. Credit card expiration.

Reasons for Voluntary Churn
-  Lack of usage
-  Poor service
-  Better price


About the data: (As provided in the overview/description)

* Customers who left within the last month – the column is called Churn
* Services that each customer has signed up for – phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movies
*  Customer account information – how long they’ve been a customer, contract, payment method, paperless billing, monthly charges, and total charges
*  Demographic info about customers – gender, age range, and if they have partners and dependents


## 1.Brief
The telecom market in the US is saturated and customer growth rates are low. They key focus of market players therefore is on retention and churn control. This project explores the churn dataset in the C50 library to identify the key drivers of churn and builds the best predictive model to predict churn. A strategy to reduce churn is presented and the proposal is evaluated against the model. Finally, the model is tuned to maximise the financial benefit of the proposed strategy.

## 2 Exploratory data analysis
The churn dataset is split into churnTrain (3333 obs.) and churnTest (1667 obs) of 19 predictor variables and 1 response variable (churn = yes/no). The proportion of churned customers (churn = yes) is close to 14% and is evenly distributed across the 2 sets.ChurnTrain will be used for data exploration and model building, while churnTest will be used to measure model performance.
About the data: 
- Services that each customer has signed up for – phone, multiple lines, internet, online security, online backup, device protection, tech support, and streaming TV and movies
- Customer account information: how long they’ve been a customer, contract, payment method, paperless billing, monthly charges, and total charges
- Demographic info about customers – gender, age range, and if they have partners and dependents



## 2.1 Churn ratio by categorical predictors

In [33]:
#Importing libraries
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Input data files are available in the "../input/" directory.
import os
import matplotlib.pyplot as plt#visualization
from PIL import  Image
%matplotlib inline
import pandas as pd
import seaborn as sns#visualization
import itertools
import warnings
warnings.filterwarnings("ignore")
import io
import plotly.offline as py#visualization
py.init_notebook_mode(connected=True)#visualization
import plotly.graph_objs as go#visualization
import plotly.tools as tls#visualization
import plotly.figure_factory as ff#visualization
#import macropy.activate 



In [22]:
telcom = pd.read_csv(r"/content/WA_Fn-UseC_-Telco-Customer-Churn.csv")
telcom.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


SyntaxError: ignored